<a href="https://colab.research.google.com/github/defneyucesir/gametheory/blob/main/Sentiment_Analysis_and_Defining_Utilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install spacy
!pip install scattertext
!pip install tika
!pip install spacytextblob
!python -m spacy download en_core_web_sm
import spacy
import json
import pylab
from IPython.core.display import display, HTML
import nltk
from tika import parser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from spacytextblob.spacytextblob import SpacyTextBlob
import requests  # Import requests for URL size checking

%matplotlib inline
pylab.rcParams['figure.figsize'] = (10., 8.)
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')
import re

def is_url_too_large(url, max_size_mb=5):
    try:
        response = requests.head(url)
        content_length = int(response.headers.get('content-length', 0))
        return content_length > max_size_mb * 1024 * 1024
    except requests.exceptions.RequestException:
        return True

def process_urls(csv):
    try:
        df = pd.read_csv(csv, encoding='utf-8-sig')
        # Rename the 'URL' column to 'url' if it exists
        if 'URL' in df.columns:
            df = df.rename(columns={'URL': 'url'})  # Rename the column here
    except:
        print(f"Error reading the file")
        return None

    # Error handling
    if 'url' not in df.columns:
        print("Error: 'url' column is required.")
        return None

    processed_urls = set()
    results = []

    for index, row in df.iterrows():
        url = row['url']  # Get the URL directly

        if url in processed_urls:
            continue

        # Using TIKA parser
        try:
            raw = parser.from_file(url)
            text = raw['content']

            # Data cleaning
            if text:
                text = " ".join(text.split())
                text = re.sub(r"[^a-zA-Z ]+", "", text)
                text = text.lower()

            # Calculate polarity and subjectivity
            if text:
                doc = nlp(text)
                polarity = doc._.blob.polarity
                subjectivity = doc._.blob.subjectivity

                # Classify sentiment based on polarity scores
                if polarity > 0.05:
                    sentiment_score_classification = "Positive"
                elif polarity < -0.05:
                    sentiment_score_classification = "Negative"
                else:
                    sentiment_score_classification = "Neutral"

                # Store results without country
                results.append([url, polarity, subjectivity, sentiment_score_classification])
                processed_urls.add(url)

        # Error handling
        except:
            print(f"Error processing URL: {url}")  # Print URL for debugging
            continue

    # Create DataFrame without country column
    new_df = pd.DataFrame(results, columns=['url', 'polarity', 'subjectivity', 'sentiment_score_classification'])
    return new_df

In [1]:
%%capture
!pip install spacy
!pip install scattertext
!pip install tika
!pip install spacytextblob
!python -m spacy download en_core_web_sm
import spacy
import json
import pylab
from IPython.core.display import display, HTML
import nltk
from tika import parser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from spacytextblob.spacytextblob import SpacyTextBlob
import requests  # Import requests for URL size checking
import traceback # Import for detailed error messages
import time

%matplotlib inline
pylab.rcParams['figure.figsize'] = (10., 8.)
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')
import re



def process_urls(csv):
    try:
        df = pd.read_csv(csv, encoding='utf-8-sig')
        # Rename the 'URL' column to 'url' if it exists
        if 'URL' in df.columns:
            df = df.rename(columns={'URL': 'url'})  # Rename the column here
    except:
        print(f"Error reading the file")
        return None

    # Error handling
    if 'url' not in df.columns:
        print("Error: 'url' column is required.")
        return None

    processed_urls = set()
    results = []

    for index, row in df.iterrows():
        url = row['url']  # Get the URL directly

        if url in processed_urls:
            continue

        # Using TIKA parser
        start_time = time.time() # Record start time
        try:
            raw = parser.from_file(url)
            text = raw['content']

            # Data cleaning
            if text:
                text = " ".join(text.split())
                text = re.sub(r"[^a-zA-Z ]+", "", text)
                text = text.lower()

            # Calculate polarity and subjectivity
            if text:
                doc = nlp(text)
                polarity = doc._.blob.polarity
                subjectivity = doc._.blob.subjectivity

                # Classify sentiment based on polarity scores
                if polarity > 0.05:
                    sentiment_score_classification = "Positive"
                elif polarity < -0.05:
                    sentiment_score_classification = "Negative"
                else:
                    sentiment_score_classification = "Neutral"

                # Store results without country
                results.append([url, polarity, subjectivity, sentiment_score_classification])
                processed_urls.add(url)

        # Error handling with detailed traceback
        except Exception as e:
            print(f"Error processing URL: {url}")  # Print URL for debugging
            print(f"Error Type: {type(e).__name__}") # Print error type
            print(f"Error Message: {e}") # Print error message
            traceback.print_exc() # Print detailed traceback
            continue # Continue to the next URL
        finally:
            end_time = time.time() # Record end time
            processing_time = end_time - start_time
            print(f"Processed URL: {url} in {processing_time:.2f} seconds") # Print processing time

    # Create DataFrame without country column
    new_df = pd.DataFrame(results, columns=['url', 'polarity', 'subjectivity', 'sentiment_score_classification'])
    return new_df

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    processed_data = process_urls(fn)

print(processed_data)

INFO:tika.tika:Retrieving https://digital-strategy.ec.europa.eu/en/policies/regulatory-framework-ai#:~:text=The%20AI%20Act%20is%20the,legal%20framework%20on%20AI%20worldwide. to /tmp/en-policies-regulatory-framework-ai.


Saving Game Theory.csv to Game Theory (11).csv


INFO:tika.tika:Retrieving https://www.gov.uk/government/publications/ai-regulation-a-pro-innovation-approach/white-paper to /tmp/government-publications-ai-regulation-a-pro-innovation-approach-white-paper.


Processed URL: https://digital-strategy.ec.europa.eu/en/policies/regulatory-framework-ai#:~:text=The%20AI%20Act%20is%20the,legal%20framework%20on%20AI%20worldwide. in 0.74 seconds


INFO:tika.tika:Retrieving https://ico.org.uk/media/about-the-ico/consultation-responses/4029424/regulating-ai-the-icos-strategic-approach.pdf to /tmp/media-about-the-ico-consultation-responses-4029424-regulating-ai-the-icos-strategic-approach.pdf.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tika/tika.py", line 778, in getRemoteFile
    urlretrieve(urlOrPath, destPath)
  File "/usr/lib/python3.11/urllib/request.py", line 241, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
                            ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
               

...

Processed URL: https://www.theguardian.com/technology/2024/mar/14/what-will-eu-proposed-regulation-ai-mean-consumers in 0.63 seconds


INFO:tika.tika:Retrieving https://iari.site/2025/03/07/eus-ai-dilemma-balancing-regulation-competitiveness-and-global-pressures/ to /tmp/2025-03-07-eus-ai-dilemma-balancing-regulation-competitiveness-and-global-pressures.


Processed URL: https://www.csis.org/analysis/japans-approach-ai-regulation-and-its-impact-2023-g7-presidency in 1.49 seconds


INFO:tika.tika:Retrieving https://www.digital.gov.au/policy/ai/policy to /tmp/policy-ai-policy.


Processed URL: https://iari.site/2025/03/07/eus-ai-dilemma-balancing-regulation-competitiveness-and-global-pressures/ in 1.89 seconds


In [ ]:
average_sentiment = processed_data['polarity'].mean()
print(f"Average Sentiment Score: {average_sentiment}")